In [ ]:
#Repetitive Sampling Plan
import numpy as np
from scipy.stats import ncx2

def main():
   while True:
        try:
            # Initial value input
            print("ENTER THE SHIFT VALUE AND NUMBER OF VARIABLES")
            D, DF = map(float, input().replace(',', ' ').split())

            print("ENTER THE INITIAL VALUES OF N, KR AND KA")
            N, KR, KA = map(float, input().replace(',', ' ').split())

            print("ENTER THE NUMBER OF ITERATIONS AND INCREMENT OF N")
            ITEN1, INCN1 = map(float, input().replace(',', ' ').split())

            print("ENTER THE NUMBER OF ITERATIONS AND INCREMENT OF KR")
            ITEKR, INCKR = map(float, input().replace(',', ' ').split())

            print("ENTER THE NUMBER OF ITERATIONS AND INCREMENT OF KA")
            ITEKA, INCKA = map(float, input().replace(',', ' ').split())

        except ValueError:
            print("Error entering values. Please make sure to provide valid numbers separated by space or comma.")
            continue

        # Iteration intervals
        N_values = N + np.arange(int(ITEN1)) * INCN1
        KA_values = KA + np.arange(int(ITEKA)) * INCKA
        KR_values = KR + np.arange(int(ITEKR)) * INCKR


        # Meshgrid to create simultaneous combinations
        N_grid, KA_grid, KR_grid = np.meshgrid(N_values, KA_values, KR_values, indexing='ij')

        # Vectorizing core calculations
        # Calculating PA1 and PR1 in vectorized form
        ALAM1 = 0
        PA1 = ncx2.cdf(KA_grid, DF, ALAM1)  # Vectorizing the function with meshgrid
        PR1 = 1 - ncx2.cdf(KR_grid, DF, ALAM1)

        # Vectorized calculations
        with np.errstate(divide='ignore', invalid='ignore'):  # Dealing with division by zero or invalid operations
            DALPHA = 1 - (PA1 / (PA1 + PR1))
            ASN1 = N_grid / (PA1 + PR1)

        # Calculation with SHIFT and the ALAM2 parameter
        ALAM2 = D * N_grid
        PA2 = ncx2.cdf(KA_grid, DF, ALAM2)
        PR2 = 1 - ncx2.cdf(KR_grid, DF, ALAM2)

        # Avoiding division by zero
        with np.errstate(divide='ignore', invalid='ignore'):
            DDBETA = PA2 / (PA2 + PR2)
            ASN2 = N_grid / (PA2 + PR2)

        DASNM = (ASN1 + ASN2) / 2

        # Optimization condition
        valid_mask = (
            (DALPHA <= 0.05) &
            (DDBETA <= 0.1) &
            (KR_grid > KA_grid)
        )

          # Applying filter
        valid_ASN1 = ASN1[valid_mask]
        valid_N_grid = N_grid[valid_mask]
        valid_KA_grid = KA_grid[valid_mask]
        valid_KR_grid = KR_grid[valid_mask]
        valid_DALPHA = DALPHA[valid_mask]
        valid_DDBETA = DDBETA[valid_mask]
        valid_ASN2 = ASN2[valid_mask]

        # Finding the best result
        if valid_ASN1.size > 0:
            idx = np.argmin(valid_ASN1)
            best_result = {
                "BN1": valid_N_grid[idx],
                "BKA": valid_KA_grid[idx],
                "BKR": valid_KR_grid[idx],
                "BASN1": valid_ASN1[idx],
                "BALPHA": valid_DALPHA[idx],
                "BBETA": valid_DDBETA[idx],
                "BASN2": valid_ASN2[idx]
            }

            # Final results
            print(f"BN1 = {best_result['BN1']:.7f}")
            print(f"BKA = {best_result['BKA']:.7f}")
            print(f"BKR = {best_result['BKR']:.7f}")
            print(f"BASN1 = {best_result['BASN1']:.7f}")
            print(f"BALPHA = {best_result['BALPHA']:.7f}")
            print(f"BBETA = {best_result['BBETA']:.7f}, ASN2 = {best_result['BASN2']:.7f}")
        else:
            print("No valid configuration found.")


if __name__ == "__main__":
    main()

In [ ]:
#Single Sampling Plan
import numpy as np
from scipy.stats import ncx2

def main():
   while True:
        try:
            # Input parameters
            print("ENTER THE SHIFT VALUE AND NUMBER OF VARIABLES (D, DF):")
            D, DF = map(float, input().replace(',', ' ').split())

            print("ENTER THE INITIAL VALUES OF N AND K:")
            N, K = map(float, input().replace(',', ' ').split())

            print("ENTER THE NUMBER OF ITERATIONS AND INCREMENT OF N (ITEN1, INCN1):")
            ITEN1, INCN1 = map(float, input().replace(',', ' ').split())

            print("ENTER THE NUMBER OF ITERATIONS AND INCREMENT OF K (ITEK, INCK):")
            ITEK, INCK = map(float, input().replace(',', ' ').split())

        except ValueError:
            print("Error entering values. Please make sure to provide valid numbers separated by space or comma.")
            continue

        # Generate values for N and K
        N_values = N + np.arange(int(ITEN1)) * INCN1
        K_values = K + np.arange(int(ITEK)) * INCK

        # Create grid for calculations
        N_grid, K_grid = np.meshgrid(N_values, K_values, indexing='ij')

        # Calculate PA1
        ALAM1 = 0
        PA1 = ncx2.cdf(K_grid, DF, ALAM1)

        # Calculate DALPHA
        DALPHA = 1 - PA1

        # Calculate PA2 using shift D
        ALAM2 = D * N_grid
        PA2 = ncx2.cdf(K_grid, DF, ALAM2)

        # Calculate DDBETA
        DDBETA = PA2

        # Apply constraints
        valid_mask = (
            (DALPHA <= 0.05) &
            (DDBETA <= 0.1)
        )

        # Filter valid configurations
        valid_N_grid = N_grid[valid_mask]
        valid_K_grid = K_grid[valid_mask]
        valid_DALPHA = DALPHA[valid_mask]
        valid_DDBETA = DDBETA[valid_mask]

        # Find minimum N
        if valid_N_grid.size > 0:
            idx = np.argmin(valid_N_grid)
            best_result = {
                "BN": valid_N_grid[idx],
                "BK": valid_K_grid[idx],
                "BALPHA": valid_DALPHA[idx],
                "BBETA": valid_DDBETA[idx],
            }

            # Output results
            print("\nOptimal Configuration Found:")
            print(f"BN = {best_result['BN']:.7f} (Minimum N)")
            print(f"BK = {best_result['BK']:.7f}")
            print(f"BALPHA = {best_result['BALPHA']:.7f}")
            print(f"BBETA = {best_result['BBETA']:.7f}\n")
        else:
            print("No valid configuration found. Please adjust input parameters.\n")

if __name__ == "__main__":
    main()